## 1️⃣ Setup & Imports

In [ ]:
# Standard imports
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add logic folder to path
project_root = Path.cwd()
logic_path = project_root / 'logic'
if str(logic_path) not in sys.path:
    sys.path.insert(0, str(logic_path))

# Import core modules
from trading_functions import unified_bayesian_gp_forecast
from alpaca_exercises import verify_alpaca_setup
from trading_assistant import trading_assistant, get_quick_health_check
from risk_management import adaptive_threshold_calculator, mcmc_optimize_thresholds
from sector_analysis import analyze_all_sectors, get_top_sectors
from math_logic.montecarlo_sims import (
    monte_carlo_strategy_simulation,
    plot_monte_carlo_results,
    print_monte_carlo_summary
)

print("✅ All modules imported successfully")

## 2️⃣ Verify Alpaca Setup

In [ ]:
# Check that Alpaca API is configured correctly
verify_alpaca_setup(verbose=True)

## 3️⃣ Define Universe & Parameters

In [ ]:
# Trading universe (10-50 stocks recommended)
UNIVERSE = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA',
    'NVDA', 'META', 'NFLX', 'AMD', 'INTC',
    'JPM', 'BAC', 'WFC', 'GS', 'MS',
    'XOM', 'CVX', 'COP', 'SLB', 'PXD'
]

# Configuration
MIN_CONF = 0.65      # Minimum confidence for trades
MIN_PROB_UP = 0.60   # Fallback ensemble threshold
MIN_Z = 0.20         # Statistical edge requirement
TP_PCT = 0.04        # Take profit: 4%
SL_PCT = 0.02        # Stop loss: 2%
MAX_ORDERS = 5       # Max positions per run
DRY_RUN = True       # Test mode

print(f"Universe: {len(UNIVERSE)} stocks")
print(f"Min Confidence: {MIN_CONF}")
print(f"TP/SL Ratio: {TP_PCT/SL_PCT:.1f}:1")
print(f"DRY_RUN: {DRY_RUN}")

## 4️⃣ Single Stock Analysis

In [ ]:
# Test forecasting on a single stock
ticker = 'AAPL'
result = unified_bayesian_gp_forecast(ticker, period='200d')

if result:
    print(f"\n🎯 Signal: {result['final_signal']}")
    print(f"📊 Confidence: {result['confidence']:.1%}")
    print(f"📈 Forecast: {result['ensemble']['forecast']:.4f}")
    print(f"🔢 Z-Score: {result['ensemble']['z_score']:.2f}")
    print(f"📉 RSI: {result['rsi']['value']:.1f}")
    print(f"📊 BB Z-Score: {result['bollinger_bands']['z_score']:.2f}")

## 5️⃣ Monte Carlo Simulation - Expected Returns

In [ ]:
# Simulate strategy performance over 30 days
mc_results = monte_carlo_strategy_simulation(
    initial_capital=100,
    avg_trades_per_day=2,
    win_rate=0.55,      # Adjust based on your observed win rate
    avg_win_pct=4.0,
    avg_loss_pct=2.0,
    days=30,
    num_simulations=1000,
    seed=42
)

# Print summary
print_monte_carlo_summary(mc_results, initial_capital=100)

# Plot results
plot_monte_carlo_results(mc_results, initial_capital=100, show=True)

## 6️⃣ Adaptive Threshold Tuning

In [ ]:
# After 20+ trades, optimize MIN_CONF dynamically
# Replace with your actual trade history
sample_trades = [
    {'confidence': 0.70, 'profit': 3.50},
    {'confidence': 0.68, 'profit': -1.80},
    {'confidence': 0.75, 'profit': 4.20},
    {'confidence': 0.72, 'profit': -2.00},
    {'confidence': 0.65, 'profit': -1.50},
    {'confidence': 0.80, 'profit': 5.00},
    {'confidence': 0.71, 'profit': -2.20},
    {'confidence': 0.69, 'profit': -1.80},
    {'confidence': 0.77, 'profit': 4.50},
    {'confidence': 0.73, 'profit': -1.90},
    {'confidence': 0.76, 'profit': 3.80},
    {'confidence': 0.68, 'profit': -2.10},
]

new_threshold = adaptive_threshold_calculator(
    trade_history=sample_trades,
    initial_min_conf=MIN_CONF,
    target_win_rate=0.55
)

print(f"\n💡 Recommended MIN_CONF: {new_threshold:.2f}")

## 7️⃣ Trading Assistant - What Should I Do?

In [ ]:
# Get automated recommendations
sample_positions = [
    {'symbol': 'AAPL', 'qty': 5, 'entry': 180.00},
    {'symbol': 'MSFT', 'qty': 3, 'entry': 370.00},
]

sample_account = {
    'cash': 50.00,
    'portfolio_value': 105.50
}

recommendations = trading_assistant(
    trade_history=sample_trades,
    current_positions=sample_positions,
    account_summary=sample_account
)

## 8️⃣ Sector Analysis & Auto-Trade

In [ ]:
# Analyze all S&P 500 sectors
sector_rankings = analyze_all_sectors(
    period='200d',
    min_confidence=0.70,
    verbose=True
)

# Get top 3 sectors
top_sectors = get_top_sectors(n=3, min_confidence=0.70)
print(f"\n🏆 Top Sectors: {top_sectors}")

## 9️⃣ MCMC Parameter Optimization (Advanced)

In [ ]:
# Optimize MIN_CONF, TP_PCT, SL_PCT using MCMC
# Requires 30+ trades and emcee package: pip install emcee

# Uncomment when you have enough trade history:
# optimal_params = mcmc_optimize_thresholds(
#     trade_history=sample_trades,
#     n_samples=5000,
#     burn_in=1000,
#     thin=10,
#     seed=42
# )
# 
# print(f"\nOptimal Parameters:")
# print(f"  MIN_CONF: {optimal_params['MIN_CONF']:.3f}")
# print(f"  TP_PCT: {optimal_params['TP_PCT']:.2f}%")
# print(f"  SL_PCT: {optimal_params['SL_PCT']:.2f}%")

print("⚠️ MCMC optimization requires 30+ trades. Collect more data first.")

## 🔟 Execute Trading Strategy (from paper_trading.ipynb)

**Note**: The execution functions (`run_once`, `summarize_universe`, etc.) are still in `paper_trading.ipynb`.  
Run that notebook to actually place trades based on these analyses.

**Daily Workflow**:
1. Run this notebook to analyze markets and get recommendations
2. Check Monte Carlo expected returns
3. Review Trading Assistant recommendations
4. Switch to `paper_trading.ipynb` and run `run_once()` to execute

---

## 📋 Summary

**Modules Created:**
- ✅ `trading_functions.py` - Bayesian/GP forecasting with Bollinger Bands
- ✅ `risk_management.py` - Adaptive thresholds, MCMC optimization
- ✅ `trading_assistant.py` - Automated performance analysis
- ✅ `sector_analysis.py` - Sector rotation strategy
- ✅ `deeplearning_nmr.py` - Deep learning integration framework
- ✅ `montecarlo_sims.py` - Strategy simulation
- ✅ `alpaca_exercises.py` - Alpaca API utilities with setup verification

**Configuration:**
- Universe: 10-50 stocks (configurable)
- Z-scores: Integrated into Bayesian model
- Bollinger Bands: Part of feature set
- MCMC: Available for parameter optimization
- Sector Analysis: Automated ranking and trading
- Trading Assistant: Tells you what to do!

**Next Steps:**
1. Collect 30+ trades for MCMC optimization
2. Implement your NMR deep learning model in `deeplearning_nmr.py`
3. Run sector analysis weekly for rotation opportunities
4. Use Trading Assistant daily for guidance

📖 **Full documentation**: `TRADING_DOCUMENTATION.md`